# Code documentation



### Overview of workflow description
  Basic file manipulation to get gnomAD allele frequency, allele count, and allele number of SNPs that are included in Zhu et. al. 2022. <br> Codes are mainly in base R. 


##### A. Getting information for gnomAD SNPs

**1. Download gnomad files and unzip**

   
> wget -q "https://storage.googleapis.com/gcp-public-data--gnomad/release/3.1.2/vcf/genomes/gnomad.genomes.v3.1.2.sites.chr1.vcf.bgz"

> gunzip -c gnomad.genomes.v3.1.2.sites.chr1.vcf.bgz > gnomad.genomes.v3.1.2.sites.chr1.vcf

**2. Extract AF, AN, AC fields of SNPs for all populations as .tsv using VCFtools**


[https://vcftools.github.io/]


> vcftools --vcf gnomad.genomes.v3.1.2.sites.chr1.vcf --out chr1VCFoutput --remove-indels --get-INFO AF_afr --get-INFO AF_asj --get-INFO AF_eas --get-INFO AF_sas --get-INFO AF_mid --get-INFO AF_fin --get-INFO AF_nfe --get-INFO AF_oth --get-INFO AF_ami --get-INFO AF_amr --get-INFO AN_afr --get-INFO AN_asj --get-INFO AN_eas --get-INFO AN_sas --get-INFO AN_mid --get-INFO AN_fin --get-INFO AN_nfe --get-INFO AN_oth --get-INFO AN_ami --get-INFO AN_amr --get-INFO AC_afr --get-INFO AC_asj --get-INFO AC_eas --get-INFO AC_sas --get-INFO AC_mid --get-INFO AC_fin --get-INFO AC_nfe --get-INFO AC_oth --get-INFO AC_ami --get-INFO AC_amr


 ##### B. Retrieving gnomAD AN, AF, AC information for UK Biobank SNPs reported in Zhu et. al. Supplementary Table S7
 
**1. T-test for sex differentiated SNPs**

 * Two sample T-test function
 * Looping script for T-test function


**2. Using liftOver to polarize the coordinates from reference genome hg19 to hg38**

 * Bash-script to generate BAM files and run liftOver
 * R function to assign new coordinates from output BAM files
 
 
 
**3. Retrieve information from tsv generated from step A**
 
 * Fetch AF, AN, and AC fields separately in three reads
 * Combine the three fields into single file
 * Flatten files
 
 
 
 

### Notes
Bolded letters are tags that are extensions to file names

##### General structure of loop scripts:

 * list.files gives a list of all files in a directory, to be used as inputs<br>
 * A for loop that feeds the index of the lists into functions
 * String splicing to generate new file name for output file


#### T-test function
  * Calculate two sample T-test based on Beta and SE values generated from GWAS data <br>
    generated using PLINK 2.0  
  * Set alpha at 0.05
  * First version of the function write output as two separate files
  * Output files have **posttests** endings

#### liftOver using UCSC command line tools

[https://genome.ucsc.edu/cgi-bin/hgLiftOver]

 * Download the template for converting
 * Download liftOver on command line
 * Bash script annotation:
   - Loop over all the **posttest** files, creating a temporary file called new.BED using the column for chromosome and position
   - Since SNP has length 1, position column is duplicated
   - Using temp file as an input to liftOver, which generates two files, the **conv_** and **unMap_**
   - Add headers to the new outputs



#### Converting coordinates

  * Function takes in three inputs: the **conv_**, **unMap_**, and original GWAS file, **_posttest**. 
  * Remove the **unMap_** SNPs, and then swap the original position columns out with new columns POS from **conv_**.
  * Since all of the reported SNPs are biallelic, the row numbers should align neatly.<br> Row numbers of **conv_** files and unMap files sum up to the row numbers of **posttest** files. 
  * Final output files gets a **coord.conv** extension
 

 

#### Match_ref and match_alt tags
 * The original GWAS beta files contains three columns per allele, REF, ALT, and A1.
 * The GWAS statistic reported is for A1 allele. 
 * The tags are there to remind readers of which allele in the site is being referred to. 
 * *match_ref* is where A1 == REF, and *match_alt* is where A1 == ALT
 * For later step, alleles with *match_ref* tags will have its values recalculated: AF = 1 - AF, and AC = AN - AC

#### Gathering AN, AC, AF information from gnomAD 

 * Nested for loops to retrieve SNPs information
 * Outer loop opens **coord.conv** file
 * Sorted chromosomal file pathnames are stored in a file named **sorted_chrom_path**
 * Inner loop iterates over all chromosomal files list and retrieve information based on chromosome, position, ref, and alt allele base
 * Create output files with ending **end_AC**, **end_AF**, or **end_AN**

 

#### Flatten all files



 * Create a list of all input files, which comes from the step above
 * Merge all AN, AF, and AC files together
 * Add a tag for T-test, "True" if pass and "False" otherwise
 * Extra step to stack all files together, such that one trait now has one file only, with **both** and **merged** tags